In [1]:
!pip install --no-cache-dir transformers sentencepiece &> /dev/null 
!pip install datasets &> /dev/null 
!pip install evaluate &> /dev/null
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 20.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import gc
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA A100-SXM4-40GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


# Load data

In [4]:
"""
data_folder = '/content/drive/MyDrive/UNLP/unlp-2023-shared-task-main/data/'
gec_fluency_train_src = data_folder + 'gec-fluency/train.src.txt'
gec_fluency_train_tgt = data_folder + 'gec-fluency/train.tgt.txt'
"""

"\ndata_folder = '/content/drive/MyDrive/UNLP/unlp-2023-shared-task-main/data/'\ngec_fluency_train_src = data_folder + 'gec-fluency/train.src.txt'\ngec_fluency_train_tgt = data_folder + 'gec-fluency/train.tgt.txt'\n"

In [4]:
#source_file = '/content/drive/MyDrive/datasets/errorified/450k-surzhik/incorr.txt'
#target_file = '/content/drive/MyDrive/datasets/errorified/450k-surzhik/corr.txt'


source_file = '/content/drive/MyDrive/UNLP/unlp-2023-shared-task-main/data/gec-fluency/train.src.txt'
target_file = '/content/drive/MyDrive/UNLP/unlp-2023-shared-task-main/data/gec-fluency/train.tgt.txt'

punct_source = "/content/drive/MyDrive/UNLP/assist-data/20k-punct-assist/source.txt"
punct_target= "/content/drive/MyDrive/UNLP/assist-data/20k-punct-assist/target.txt"

dilute_source = "/content/drive/MyDrive/artem-yushko/data-artem/cleaned/borshch4.txt"
dilute_target = "/content/drive/MyDrive/artem-yushko/data-artem/cleaned/borshch4.txt"

In [5]:
import pandas as pd
def read_parallel_into_df(source_file, target_file):
  # reading the input data
  with open(source_file, 'r') as f:
      source = [line[:-1] for line in f.readlines()]

  with open(target_file, 'r') as f:
      target = [line[:-1] for line in f.readlines()]
  
  all_sentences = [[source[i], target[i]] for i in range(len(source))]
  df = pd.DataFrame(all_sentences)
  df.columns = ['source', 'target']
  return df

In [35]:
ua_gec_df = read_parallel_into_df(source_file, target_file)
#punct_df = read_parallel_into_df(punct_source, punct_target)
#dilute = read_parallel_into_df(dilute_source, dilute_target)
#dilute_curr = dilute[:100000]

In [32]:
with open( '/content/drive/MyDrive/UNLP/unlp-2023-shared-task-main/data/gec-fluency/train.m2', 'r') as f:
  sentences = f.read().split('\n\n')[:-1]

sentence_split = []
for sentence in sentences:
  sentence_split.append(sentence.split('\n'))

sent_df = []
for sent in sentence_split:

  edits = []
  for edit in sent[1:]:
    edit_lst = edit.split('|||')
    edit_type = edit_lst[1]
    ref = edit_lst[-1]

    edits.append([edit_type, ref])
  sent_dict = {'sent':sent[0], 
               'all':sent, 
               'total-edits':len(sent)-1,
               'all-edits':edits}
  sent_df.append(sent_dict)
sent_df = pd.DataFrame(sent_df)

big_edits = []
for edits in sent_df['all-edits']:
  for edit in edits:
    big_edits.append(edit[0])

all_cats = list(set(big_edits))

for cat in all_cats:
  sent_df[cat] = 0

sent_df['ref0'] = 0
sent_df['ref1'] = 0

for i in range(len(sent_df)):
  edits = sent_df['all-edits'][i]
  for edit in edits:
    sent_df[edit[0]][i] += 1
    sent_df[f'ref{edit[1]}'][i] +=1

drop_idx = list(sent_df[sent_df['total-edits']>6].index)

<ipython-input-32-52a0a700dd96>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sent_df[edit[0]][i] += 1
<ipython-input-32-52a0a700dd96>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sent_df[f'ref{edit[1]}'][i] +=1


In [51]:
sent_df

,sent,all,total-edits,all-edits,G/Conjunction,G/PartVoice,G/UngrammaticalStructure,F/PoorFlow,noop,Spelling,...,Other,G/VerbVoice,F/Collocation,F/Repetition,G/Other,F/Other,G/Comparison,G/Prep,ref0,ref1
0,S # 0000,"[S # 0000, A -1 -1|||noop|||-NONE-|||REQUIRED|...",2,"[[noop, 0], [noop, 1]]",0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,1
1,S Byte for France або “ Мій досвід ведення бло...,[S Byte for France або “ Мій досвід ведення бл...,2,"[[Spelling, 0], [Spelling, 1]]",0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,1,1
2,S Останні 3 місяці мого життя видалися аж зана...,[S Останні 3 місяці мого життя видалися аж зан...,7,"[[F/Style, 0], [Punctuation, 0], [F/Style, 0],...",0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,3,4
3,S Сьогодні розповім про те як і навіщо мене за...,[S Сьогодні розповім про те як і навіщо мене з...,3,"[[Punctuation, 0], [Spelling, 0], [Punctuation...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2,1
4,S Якщо цікаво подивитися відразу на результат ...,[S Якщо цікаво подивитися відразу на результат...,3,"[[Punctuation, 0], [F/Style, 1], [Punctuation,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32729,S # 1889,"[S # 1889, A -1 -1|||noop|||-NONE-|||REQUIRED|...",2,"[[noop, 0], [noop, 1]]",0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,1
32730,S Яким боком московити нам брати ?,"[S Яким боком московити нам брати ?, A 0 2|||F...",2,"[[F/Style, 0], [Punctuation, 0]]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
32731,"S І навіть якщо брати , то Авель теж мав брата .","[S І навіть якщо брати , то Авель теж мав брат...",1,"[[noop, 0]]",0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
32732,S # 1890,"[S # 1890, A -1 -1|||noop|||-NONE-|||REQUIRED|...",2,"[[noop, 0], [noop, 1]]",0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,1


In [36]:
len(drop_idx)

472

In [37]:
for id in drop_idx:
  ua_gec_df = ua_gec_df.drop(id, axis=0)
len(ua_gec_df)

32262

In [38]:
train_df = pd.concat([ua_gec_df])

# converting pandas dataframe to Dataset class
from datasets import Dataset
raw_dataset = Dataset.from_pandas(train_df)

In [39]:
raw_dataset

Dataset({
    features: ['source', 'target', '__index_level_0__'],
    num_rows: 32262
})

# Train

In [40]:
# SET MODEL NAME
model_name = 'unlp-drop-f-errors'

In [41]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="uk_UA", tgt_lang="uk_UA")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--mbart-large-50/snapshots/85b0b6b66d2330ba1ad2da18dd3823f8451a929b/config.json
Model config MBartConfig {
  "_name_or_path": "facebook/mbart-large-50",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
   

In [42]:
ukr_sent_src = "Привіт андрій як справи"
ukr_sent_tgt = "Привіт, Андрію, як справи?"

inputs = tokenizer(ukr_sent_src, text_target=ukr_sent_tgt)
print(inputs)
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

{'input_ids': [250048, 1813, 24046, 6, 144279, 2800, 1443, 43282, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [250048, 1813, 24046, 4, 43760, 10506, 4, 1443, 43282, 32, 2]}
['uk_UA', '▁При', 'віт', '▁', 'андр', 'ій', '▁як', '▁справи', '</s>']
['uk_UA', '▁При', 'віт', ',', '▁Андр', 'ію', ',', '▁як', '▁справи', '?', '</s>']


In [43]:
max_length = 128

def preprocess_function(examples):
    inputs = examples["source"]
    targets = examples["target"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [44]:
tokenized_datasets = raw_dataset.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/32262 [00:00<?, ? examples/s]

In [45]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--mbart-large-50/snapshots/85b0b6b66d2330ba1ad2da18dd3823f8451a929b/config.json
Model config MBartConfig {
  "_name_or_path": "facebook/mbart-large-50",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
   

In [46]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [47]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/lenguist/unlp-drop-total-edits-6up into local empty directory.
Using cuda_amp half precision backend


In [49]:
torch.cuda.empty_cache()

In [50]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: __index_level_0__, source, target. If __index_level_0__, source, target are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 32262
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3027
  Number of trainable parameters = 610879488
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast token

Step,Training Loss
500,0.515900
1000,0.256800
1500,0.161900
2000,0.157800
2500,0.113700
3000,0.101900


Saving model checkpoint to unlp-drop-total-edits-6up/checkpoint-1009
Configuration saved in unlp-drop-total-edits-6up/checkpoint-1009/config.json
Configuration saved in unlp-drop-total-edits-6up/checkpoint-1009/generation_config.json
Model weights saved in unlp-drop-total-edits-6up/checkpoint-1009/pytorch_model.bin
tokenizer config file saved in unlp-drop-total-edits-6up/checkpoint-1009/tokenizer_config.json
Special tokens file saved in unlp-drop-total-edits-6up/checkpoint-1009/special_tokens_map.json
tokenizer config file saved in unlp-drop-total-edits-6up/tokenizer_config.json
Special tokens file saved in unlp-drop-total-edits-6up/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
Saving model checkpoint to unlp-drop-total-edits-6up/checkpoint-2018
Configuration saved in unlp-drop-total-edits-6up/checkpoint-2018/config.json
Configuration saved in unlp-drop-total-edits-6up/checkpoint-2018/generation_config.j

TrainOutput(global_step=3027, training_loss=0.21696207150792304, metrics={'train_runtime': 974.8387, 'train_samples_per_second': 99.284, 'train_steps_per_second': 3.105, 'total_flos': 1.4565393452531712e+16, 'train_loss': 0.21696207150792304, 'epoch': 3.0})

#MT5 experiments

In [ ]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")


/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
text_src = "Привіт світ"
text_tgt = "Привіт світe"
inputs = tokenizer(text_src, text_target=text_tgt, return_tensors="pt")

outputs = model(**inputs)
loss = outputs.loss

In [ ]:
from transformers import pipeline
translator = pipeline("text-generation", model= "google/mt5-base", device=0)

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The model 'MT5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GP

In [ ]:
translator("Привіт світ")

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Привіт світ>... <extra_id_1>? <extra_id_1>? <extra_id_29>lêdobitlivezaítвид Iвидi...'}]